In [1]:
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Cloning into 'LightGBM'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 20602 (delta 11), reused 0 (delta 0), pack-reused 20570
Receiving objects: 100% (20602/20602), 16.41 MiB | 34.30 MiB/s, done.
Resolving deltas: 100% (15053/15053), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728), 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
from google.colab import drive
drive.mount('/content/drive')
from datetime import date
import lightgbm as lgbm
from lightgbm import LGBMRegressor
import time
import statsmodels.api as sm
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.regression.quantile_regression import QuantReg
from sklearn.ensemble import GradientBoostingRegressor
import os

Mounted at /content/drive


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [154]:
import math
from math import radians
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import curve_fit
import pylab

In [4]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 3                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 3
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_left = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_left)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [5]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')
submission.shape

(7776, 10)

In [6]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)

  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)
  return dataset

In [7]:
def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

In [8]:
def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

In [47]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def train_to_supervised(train, target, n_in):

    clmns = list(train.columns)

    # 기타 칼럼은 전과 같이 들어갈 것.
    scaled_lst = clmns

    scaled_df = train[scaled_lst]
    target_df = target

    # 미래 몇 번째 항목을 가져올 것인가
    future = [48, 96]

    ### 만약에 스케일링을 하고 싶다면 ###
    # scaled_df 데이터 프레임만 스케일링 하고, 절기랑 TARGET 데이터는 그냥 두면 된다.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    n_vars2 = 1 if type(target_df) is list else target_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # 48과 96 후의 타겟 데이터 2개 붙이기.
    # forecast sequence (t, t+1, ... t+n)
    for i in future:
        cols.append(target_df.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [48]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def test_to_supervised(train, n_in):

    clmns = list(train.columns)

    # 타켓 칼럼의 이름을 여기에 입력
    target = ['TARGET']

    # 클래스 변수로 전환(encoding)할 칼럼을 여기에 입력
#    class_lst = ['season']

    # 기타 칼럼은 전과 같이 들어갈 것.
#    scaled_lst = list(set(clmns) - set(class_lst))

    scaled_df = train.copy()
#    class_df = train[class_lst]

    ### 만약에 스케일링을 하고 싶다면 ###
    # testset의 스케일링은 구조가 상당히 까다로우므로....... 일단 나중에 하기로 함.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # class라 encoding 필요한 절기 끝에 하나만 붙여놓기
#    cols.append(class_df)
#    names += class_lst
    
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [49]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]

In [50]:
m = 60

일출 = [7+47/m, 7+44/m, 7+34/m, 7+18/m, 6+56/m, 6+35/m, 6+12/m, 5+51/m, 5+33/m, 5+19/m, 5+11/m, 5+11/m, 5+17/m, 5+27/m, 5+41/m, 5+54/m, 6+8/m, 6+20/m, 6+33/m, 6+47/m, 7+3/m, 7+20/m, 7+33/m, 7+45/m]
일몰 = [17+28/m, 17+42/m, 17+58/m, 18+15/m, 18+30/m, 18+44/m, 18+58/m, 19+12/m, 19+25/m, 19+39/m, 19+50/m, 19+57/m, 19+56/m, 19+50/m, 19+35/m, 19+15/m, 18+51/m, 18+28/m, 18+6/m, 17+45/m, 17+28/m, 17+17/m, 17+13/m, 17+17/m]
경사각 = [32.92, 36.83, 40.75, 44.67, 48.58, 52.5, 56.42, 60.33, 64.25, 68.16, 72.01, 76, 72.1, 68.16, 64.25, 60.33, 56.42, 52.5, 48.58, 44.67, 40.75, 36.83, 32.92, 29]
남중 = [12+36/m, 12+41/m, 12+44/m, 12+44/m, 12+42/m, 12+38/m, 12+34/m, 12+29/m, 12+27/m, 12+27/m, 12+29/m, 12+32/m, 12+35/m, 12+37/m, 12+36/m, 12+33/m, 12+29/m, 12+23/m, 12+18/m, 12+15/m, 12+14/m, 12+16/m, 12+22/m, 12+29/m]
print(len(일출), len(일몰), len(경사각), len(남중))

24 24 24 24


In [51]:
def HRA(DHI, DNI, season, hour):
  # 위도(latitude) 기준을 일단 임의로 대전으로 설정 (위도 36.19~36.2도)
  latitude = radians(36.2)
  season = int(season)
  # 절기별 대한민국의 경사각
  tilt = radians(경사각[season])

  # 절기별 대한민국 대전의 태양 남중시각
  hra = radians(15*(hour - 남중[season]))

  # 구하려는 알파
  elevation = np.arcsin(np.sin(tilt) * np.sin(latitude) + np.cos(tilt) * np.cos(latitude) * np.cos(hra))

  # 천정각(Zenith Angle)은 90 - 알파
  zenith = radians(90) - elevation

  # GHI는 DHI + DNI * cos(천정각)
  ghi = DHI + DNI *np.cos(zenith)

  return ghi

In [52]:
def encoding(data):
  season_list = [i for i in range(0,24)]
  for k in season_list:
    data['Season_' +f'{k}'] = data['season'] == k
  return data*1

In [53]:
def not_minus(dataset):
  for i in range(0, len(dataset.index)):
    for j in range(0, len(dataset.columns)):
      K = dataset.iloc[i,j]
      if K < 0.09:
        dataset.iloc[i,j] = 0
  return dataset

In [54]:
dataset = raw_data.copy()
dataset2 = drop_clms(dataset)
dataset2.drop(['Hour', 'Minute'], axis =1, inplace=True)
dataset2.rename(columns = {'T' : 'Temp'}, inplace = True)
dataset2

,DHI,DNI,WS,RH,Temp,TARGET,Time,Date
0,0,0,1.5,69.08,-12,0.0,0.0,0
1,0,0,1.5,69.06,-12,0.0,0.5,0
2,0,0,1.6,71.78,-12,0.0,1.0,0
3,0,0,1.6,71.75,-12,0.0,1.5,0
4,0,0,1.6,75.20,-12,0.0,2.0,0
...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,21.5,364
52556,0,0,2.4,66.79,-4,0.0,22.0,364
52557,0,0,2.2,66.78,-4,0.0,22.5,364
52558,0,0,2.1,67.72,-4,0.0,23.0,364


In [55]:
dataset2['season'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)
dataset3 = cos_time(dataset2)
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'Temp', 'TARGET', 'sin_time', 'cos_time','season', 'Time']]
# dataset5 = dataset4.iloc[n_days:,:-1]
# goals = pd.DataFrame(dataset4.Goal, index = dataset4.index)
dataset4

,DHI,DNI,WS,RH,Temp,TARGET,sin_time,cos_time,season,Time
0,0,0,1.5,69.08,-12,0.0,0.000000,1.000000,23,0.0
1,0,0,1.5,69.06,-12,0.0,0.130526,0.991445,23,0.5
2,0,0,1.6,71.78,-12,0.0,0.258819,0.965926,23,1.0
3,0,0,1.6,71.75,-12,0.0,0.382683,0.923880,23,1.5
4,0,0,1.6,75.20,-12,0.0,0.500000,0.866025,23,2.0
...,...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,-0.608761,0.793353,0,21.5
52556,0,0,2.4,66.79,-4,0.0,-0.500000,0.866025,0,22.0
52557,0,0,2.2,66.78,-4,0.0,-0.382683,0.923880,0,22.5
52558,0,0,2.1,67.72,-4,0.0,-0.258819,0.965926,0,23.0


In [56]:
dataset5= dataset4.copy()

dataset5['GHIs'] = dataset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [57]:
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,Temp,TARGET,sin_time,cos_time,season,Time,GHIs
50,0,0,1.6,88.11,-11,0.0,0.258819,0.965926,23,1.0,0.0


In [58]:
def add_variables(dataset):
  sunny = list()
  avg_temp = list()
  max_temp = list()
  min_temp = list()
  avg_ghi = list()
  max_ghi = list()
  min_ghi = list()
  avg_tar = list()
  max_tar = list()
  min_tar = list()


  for i in range(0, len(dataset), 48):
    temp = dataset['Temp'][i:i+48]
    ghi = dataset['GHIs'][i:i+48]
    tar = dataset['TARGET'][i:i+48]
    sunny += [24-(list(dataset.DHI[i:i+48]).count(0)*0.5)]*48 
    avg_temp += [temp.mean()]*48
    max_temp += [temp.max()]*48
    min_temp += [temp.min()]*48
    avg_ghi += [ghi.sum()/(24-(list(dataset.DHI[i:i+48]).count(0)*0.5))]*48
    max_ghi += [ghi.max()]*48
    min_ghi += [ghi[ghi>0.01].min()]*48
    avg_tar += [tar.sum()/(24-(list(dataset.DHI[i:i+48]).count(0)*0.5))]*48
    max_tar += [tar.max()]*48
    min_tar += [tar[tar>0.01].min()]*48


  dataset["SH"] = sunny
  dataset["Avg_temp"] = avg_temp
  dataset['Max_temp'] = max_temp
  dataset['Min_temp'] = min_temp
  dataset["Avg_GHI"] = avg_ghi
  dataset['Max_GHI'] = max_ghi
  dataset['Min_GHI'] = min_ghi
  dataset["Avg_TAR"] = avg_tar
  dataset['Max_TAR'] = max_tar
  dataset['Min_TAR'] = min_tar
  
  return dataset

In [59]:
dataset6 = add_variables(dataset5)
dataset6

,DHI,DNI,WS,RH,Temp,TARGET,sin_time,cos_time,season,Time,GHIs,SH,Avg_temp,Max_temp,Min_temp,Avg_GHI,Max_GHI,Min_GHI,Avg_TAR,Max_TAR,Min_TAR
0,0,0,1.5,69.08,-12,0.0,0.000000,1.000000,23,0.0,0.0,9.0,-7.979167,-3,-12,578.031164,705.717934,10.000000,34.777337,33.129393,0.938541
1,0,0,1.5,69.06,-12,0.0,0.130526,0.991445,23,0.5,0.0,9.0,-7.979167,-3,-12,578.031164,705.717934,10.000000,34.777337,33.129393,0.938541
2,0,0,1.6,71.78,-12,0.0,0.258819,0.965926,23,1.0,0.0,9.0,-7.979167,-3,-12,578.031164,705.717934,10.000000,34.777337,33.129393,0.938541
3,0,0,1.6,71.75,-12,0.0,0.382683,0.923880,23,1.5,0.0,9.0,-7.979167,-3,-12,578.031164,705.717934,10.000000,34.777337,33.129393,0.938541
4,0,0,1.6,75.20,-12,0.0,0.500000,0.866025,23,2.0,0.0,9.0,-7.979167,-3,-12,578.031164,705.717934,10.000000,34.777337,33.129393,0.938541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,-0.608761,0.793353,0,21.5,0.0,9.0,-2.708333,1,-5,1024.965465,771.947011,169.653975,46.454798,39.322261,2.252448
52556,0,0,2.4,66.79,-4,0.0,-0.500000,0.866025,0,22.0,0.0,9.0,-2.708333,1,-5,1024.965465,771.947011,169.653975,46.454798,39.322261,2.252448
52557,0,0,2.2,66.78,-4,0.0,-0.382683,0.923880,0,22.5,0.0,9.0,-2.708333,1,-5,1024.965465,771.947011,169.653975,46.454798,39.322261,2.252448
52558,0,0,2.1,67.72,-4,0.0,-0.258819,0.965926,0,23.0,0.0,9.0,-2.708333,1,-5,1024.965465,771.947011,169.653975,46.454798,39.322261,2.252448


In [60]:
#dataset6 = dataset5[['DHI', 'DNI', 'WS', 'RH', 'Temp','SH', 'sin_time', 'cos_time', 'GHI', 'season', 'TARGET']]
dataset7 = dataset6[['RH', 'Temp','Avg_temp', 'Max_temp', 'Min_temp', 'Avg_GHI', 'Max_GHI','Min_GHI', 'GHIs', 'Avg_TAR', 'Max_TAR', 'Min_TAR', 'TARGET']] # 'SH', 'sin_time', 'cos_time', 
dataset7.head(10)

,RH,Temp,Avg_temp,Max_temp,Min_temp,Avg_GHI,Max_GHI,Min_GHI,GHIs,Avg_TAR,Max_TAR,Min_TAR,TARGET
0,69.08,-12,-7.979167,-3,-12,578.031164,705.717934,10.0,0.0,34.777337,33.129393,0.938541,0.0
1,69.06,-12,-7.979167,-3,-12,578.031164,705.717934,10.0,0.0,34.777337,33.129393,0.938541,0.0
2,71.78,-12,-7.979167,-3,-12,578.031164,705.717934,10.0,0.0,34.777337,33.129393,0.938541,0.0
3,71.75,-12,-7.979167,-3,-12,578.031164,705.717934,10.0,0.0,34.777337,33.129393,0.938541,0.0
4,75.20,-12,-7.979167,-3,-12,578.031164,705.717934,10.0,0.0,34.777337,33.129393,0.938541,0.0
5,69.29,-11,-7.979167,-3,-12,578.031164,705.717934,10.0,0.0,34.777337,33.129393,0.938541,0.0
6,72.56,-11,-7.979167,-3,-12,578.031164,705.717934,10.0,0.0,34.777337,33.129393,0.938541,0.0
7,72.55,-11,-7.979167,-3,-12,578.031164,705.717934,10.0,0.0,34.777337,33.129393,0.938541,0.0
8,74.62,-11,-7.979167,-3,-12,578.031164,705.717934,10.0,0.0,34.777337,33.129393,0.938541,0.0
9,74.61,-11,-7.979167,-3,-12,578.031164,705.717934,10.0,0.0,34.777337,33.129393,0.938541,0.0


In [61]:
scaler = MinMaxScaler()
# scaler = StandardScaler()
scaler.fit(dataset7)
temp_X = pd.DataFrame(scaler.transform(dataset7), columns = dataset7.columns)
temp_y = pd.DataFrame(dataset7['TARGET'])

## dataset6 가지고 동일하게 to_supervised 함수를 써서 나누면 됩니다.
### 나머지 변수 합치기, 인코딩은 유진이가 해줄 것...

In [62]:
dataset8 = train_to_supervised(temp_X,temp_y, n_days)
dataset8

,RH(t-144),Temp(t-144),Avg_temp(t-144),Max_temp(t-144),Min_temp(t-144),Avg_GHI(t-144),Max_GHI(t-144),Min_GHI(t-144),GHIs(t-144),Avg_TAR(t-144),Max_TAR(t-144),Min_TAR(t-144),TARGET(t-144),RH(t-143),Temp(t-143),Avg_temp(t-143),Max_temp(t-143),Min_temp(t-143),Avg_GHI(t-143),Max_GHI(t-143),Min_GHI(t-143),GHIs(t-143),Avg_TAR(t-143),Max_TAR(t-143),Min_TAR(t-143),TARGET(t-143),RH(t-142),Temp(t-142),Avg_temp(t-142),Max_temp(t-142),Min_temp(t-142),Avg_GHI(t-142),Max_GHI(t-142),Min_GHI(t-142),GHIs(t-142),Avg_TAR(t-142),Max_TAR(t-142),Min_TAR(t-142),TARGET(t-142),RH(t-141),...,Temp(t-3),Avg_temp(t-3),Max_temp(t-3),Min_temp(t-3),Avg_GHI(t-3),Max_GHI(t-3),Min_GHI(t-3),GHIs(t-3),Avg_TAR(t-3),Max_TAR(t-3),Min_TAR(t-3),TARGET(t-3),RH(t-2),Temp(t-2),Avg_temp(t-2),Max_temp(t-2),Min_temp(t-2),Avg_GHI(t-2),Max_GHI(t-2),Min_GHI(t-2),GHIs(t-2),Avg_TAR(t-2),Max_TAR(t-2),Min_TAR(t-2),TARGET(t-2),RH(t-1),Temp(t-1),Avg_temp(t-1),Max_temp(t-1),Min_temp(t-1),Avg_GHI(t-1),Max_GHI(t-1),Min_GHI(t-1),GHIs(t-1),Avg_TAR(t-1),Max_TAR(t-1),Min_TAR(t-1),TARGET(t-1),TARGET1(t+48),TARGET1(t+96)
144,0.665404,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.665188,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.694622,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.694297,...,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.0,0.071674,0.078569,0.075271,0.0,0.795368,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.0,0.071674,0.078569,0.075271,0.0,0.795260,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.0,0.071674,0.078569,0.075271,0.0,0.0,0.0
145,0.665188,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.694622,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.694297,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.731631,...,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.0,0.071674,0.078569,0.075271,0.0,0.795260,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.0,0.071674,0.078569,0.075271,0.0,0.759333,0.092593,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.0,0.198174,0.309021,0.032260,0.0,0.0,0.0
146,0.694622,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.694297,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.731631,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.667677,...,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.0,0.071674,0.078569,0.075271,0.0,0.759333,0.092593,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.0,0.198174,0.309021,0.032260,0.0,0.759442,0.111111,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.0,0.198174,0.309021,0.032260,0.0,0.0,0.0
147,0.694297,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.731631,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.667677,0.148148,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.703062,...,0.092593,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.0,0.198174,0.309021,0.032260,0.0,0.759442,0.111111,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.0,0.198174,0.309021,0.032260,0.0,0.758576,0.111111,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.0,0.198174,0.309021,0.032260,0.0,0.0,0.0
148,0.731631,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.667677,0.148148,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.703062,0.148148,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0

In [63]:
X = dataset8.iloc[:,:-2]
#X = encoding(X)
#X.drop('season', axis =1, inplace=True)
y = pd.DataFrame(dataset8.iloc[:,-2:])
X

,RH(t-144),Temp(t-144),Avg_temp(t-144),Max_temp(t-144),Min_temp(t-144),Avg_GHI(t-144),Max_GHI(t-144),Min_GHI(t-144),GHIs(t-144),Avg_TAR(t-144),Max_TAR(t-144),Min_TAR(t-144),TARGET(t-144),RH(t-143),Temp(t-143),Avg_temp(t-143),Max_temp(t-143),Min_temp(t-143),Avg_GHI(t-143),Max_GHI(t-143),Min_GHI(t-143),GHIs(t-143),Avg_TAR(t-143),Max_TAR(t-143),Min_TAR(t-143),TARGET(t-143),RH(t-142),Temp(t-142),Avg_temp(t-142),Max_temp(t-142),Min_temp(t-142),Avg_GHI(t-142),Max_GHI(t-142),Min_GHI(t-142),GHIs(t-142),Avg_TAR(t-142),Max_TAR(t-142),Min_TAR(t-142),TARGET(t-142),RH(t-141),...,TARGET(t-4),RH(t-3),Temp(t-3),Avg_temp(t-3),Max_temp(t-3),Min_temp(t-3),Avg_GHI(t-3),Max_GHI(t-3),Min_GHI(t-3),GHIs(t-3),Avg_TAR(t-3),Max_TAR(t-3),Min_TAR(t-3),TARGET(t-3),RH(t-2),Temp(t-2),Avg_temp(t-2),Max_temp(t-2),Min_temp(t-2),Avg_GHI(t-2),Max_GHI(t-2),Min_GHI(t-2),GHIs(t-2),Avg_TAR(t-2),Max_TAR(t-2),Min_TAR(t-2),TARGET(t-2),RH(t-1),Temp(t-1),Avg_temp(t-1),Max_temp(t-1),Min_temp(t-1),Avg_GHI(t-1),Max_GHI(t-1),Min_GHI(t-1),GHIs(t-1),Avg_TAR(t-1),Max_TAR(t-1),Min_TAR(t-1),TARGET(t-1)
144,0.665404,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.665188,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.694622,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.694297,...,0.0,0.773618,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.0,0.071674,0.078569,0.075271,0.0,0.795368,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.0,0.071674,0.078569,0.075271,0.0,0.795260,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.0,0.071674,0.078569,0.075271,0.0
145,0.665188,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.694622,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.694297,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.731631,...,0.0,0.795368,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.0,0.071674,0.078569,0.075271,0.0,0.795260,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.0,0.071674,0.078569,0.075271,0.0,0.759333,0.092593,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.0,0.198174,0.309021,0.032260,0.0
146,0.694622,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.694297,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.731631,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.667677,...,0.0,0.795260,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.0,0.071674,0.078569,0.075271,0.0,0.759333,0.092593,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.0,0.198174,0.309021,0.032260,0.0,0.759442,0.111111,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.0,0.198174,0.309021,0.032260,0.0
147,0.694297,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.731631,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.667677,0.148148,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.703062,...,0.0,0.759333,0.092593,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.0,0.198174,0.309021,0.032260,0.0,0.759442,0.111111,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.0,0.198174,0.309021,0.032260,0.0,0.758576,0.111111,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.0,0.198174,0.309021,0.032260,0.0
148,0.731631,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.667677,0.148148,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.0,0.232124,0.254563,0.096779,0.0,0.703062,0.148148,0.153525,0.173913,0.189189,0.299808,0.585426,0.

In [64]:
X_train, X_valid, Y_train,  Y_valid = train_test_split(X, y, test_size=0.30, random_state=42)

# testset은 이미 24절기(season)가 들어있으므로, 약간 다르게 전처리를 해 줘야 함.
## (season) 구하는 과정이 생략됨.
## 이하는 testset을 구하는 코드

In [65]:
file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(1) + '.csv'
new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(1) + '.csv'
file_name = str(1) + '.csv'

# adj_test에서 가져와야 하므로 new_path에서 불러온다
temp = pd.read_csv(new_path)
temp

,Unnamed: 0,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,Time,24
0,0,0,0,0,0,0,2.8,70.34,-6.1,0.0,0.0,22
1,1,0,0,30,0,0,2.9,71.97,-6.4,0.0,0.5,22
2,2,0,1,0,0,0,2.9,69.77,-6.7,0.0,1.0,22
3,3,0,1,30,0,0,2.9,71.95,-7.1,0.0,1.5,22
4,4,0,2,0,0,0,2.9,69.48,-7.4,0.0,2.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...
331,331,6,21,30,0,0,1.7,43.65,-5.2,0.0,21.5,22
332,332,6,22,0,0,0,1.5,40.62,-4.9,0.0,22.0,22
333,333,6,22,30,0,0,1.5,40.62,-4.9,0.0,22.5,22
334,334,6,23,0,0,0,1.5,39.21,-4.9,0.0,23.0,22


In [66]:
temp.drop('Unnamed: 0', axis = 1, inplace = True)

In [67]:
testset = temp.copy()

# 24로 저장한 24절기 season으로 이름 바꾸기
testset.rename(columns = {'24' : 'season', 'T' : 'Temp'}, inplace = True)


# 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
testset2 = drop_clms(testset)
testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
testset3 = cos_time(testset2)
testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'Temp', 'TARGET', 'sin_time', 'cos_time','season', 'Time']]
testset4

,DHI,DNI,WS,RH,Temp,TARGET,sin_time,cos_time,season,Time
0,0,0,2.8,70.34,-6.1,0.0,0.000000,1.000000,22,0.0
1,0,0,2.9,71.97,-6.4,0.0,0.130526,0.991445,22,0.5
2,0,0,2.9,69.77,-6.7,0.0,0.258819,0.965926,22,1.0
3,0,0,2.9,71.95,-7.1,0.0,0.382683,0.923880,22,1.5
4,0,0,2.9,69.48,-7.4,0.0,0.500000,0.866025,22,2.0
...,...,...,...,...,...,...,...,...,...,...
331,0,0,1.7,43.65,-5.2,0.0,-0.608761,0.793353,22,21.5
332,0,0,1.5,40.62,-4.9,0.0,-0.500000,0.866025,22,22.0
333,0,0,1.5,40.62,-4.9,0.0,-0.382683,0.923880,22,22.5
334,0,0,1.5,39.21,-4.9,0.0,-0.258819,0.965926,22,23.0


In [68]:
testset5 = testset4.copy()
testset5['GHIs'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [69]:
testset5.iloc[50:51,:]

,DHI,DNI,WS,RH,Temp,TARGET,sin_time,cos_time,season,Time,GHIs
50,0,0,2.1,61.33,-10.8,0.0,0.258819,0.965926,22,1.0,0.0


In [70]:
#### testset과 dataset이 같은 모양인지 중간점검
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,Temp,TARGET,sin_time,cos_time,season,Time,GHIs,SH,Avg_temp,Max_temp,Min_temp,Avg_GHI,Max_GHI,Min_GHI,Avg_TAR,Max_TAR,Min_TAR
50,0,0,1.6,88.11,-11,0.0,0.258819,0.965926,23,1.0,0.0,9.0,-6.3125,2,-12,1466.243756,1036.281607,23.0,58.800983,48.612666,2.158549


In [74]:
testset6 = add_variables(testset5)

In [75]:
testset6 = testset5[['RH', 'Temp','Avg_temp', 'Max_temp', 'Min_temp', 'Avg_GHI', 'Max_GHI','Min_GHI', 'GHIs', 'Avg_TAR', 'Max_TAR', 'Min_TAR', 'TARGET']]
testset6.head(10)

,RH,Temp,Avg_temp,Max_temp,Min_temp,Avg_GHI,Max_GHI,Min_GHI,GHIs,Avg_TAR,Max_TAR,Min_TAR,TARGET
0,70.34,-6.1,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,0.0,36.177302,40.637337,0.656981,0.0
1,71.97,-6.4,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,0.0,36.177302,40.637337,0.656981,0.0
2,69.77,-6.7,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,0.0,36.177302,40.637337,0.656981,0.0
3,71.95,-7.1,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,0.0,36.177302,40.637337,0.656981,0.0
4,69.48,-7.4,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,0.0,36.177302,40.637337,0.656981,0.0
5,72.13,-7.9,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,0.0,36.177302,40.637337,0.656981,0.0
6,69.76,-8.3,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,0.0,36.177302,40.637337,0.656981,0.0
7,71.97,-8.7,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,0.0,36.177302,40.637337,0.656981,0.0
8,68.56,-9.1,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,0.0,36.177302,40.637337,0.656981,0.0
9,70.75,-9.5,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,0.0,36.177302,40.637337,0.656981,0.0


In [76]:
testset6 = pd.DataFrame(scaler.transform(testset6), columns = testset6.columns)

In [77]:
testset7 = test_to_supervised(testset6, n_days)
testset7

,RH(t-144),Temp(t-144),Avg_temp(t-144),Max_temp(t-144),Min_temp(t-144),Avg_GHI(t-144),Max_GHI(t-144),Min_GHI(t-144),GHIs(t-144),Avg_TAR(t-144),Max_TAR(t-144),Min_TAR(t-144),TARGET(t-144),RH(t-143),Temp(t-143),Avg_temp(t-143),Max_temp(t-143),Min_temp(t-143),Avg_GHI(t-143),Max_GHI(t-143),Min_GHI(t-143),GHIs(t-143),Avg_TAR(t-143),Max_TAR(t-143),Min_TAR(t-143),TARGET(t-143),RH(t-142),Temp(t-142),Avg_temp(t-142),Max_temp(t-142),Min_temp(t-142),Avg_GHI(t-142),Max_GHI(t-142),Min_GHI(t-142),GHIs(t-142),Avg_TAR(t-142),Max_TAR(t-142),Min_TAR(t-142),TARGET(t-142),RH(t-141),...,TARGET(t-4),RH(t-3),Temp(t-3),Avg_temp(t-3),Max_temp(t-3),Min_temp(t-3),Avg_GHI(t-3),Max_GHI(t-3),Min_GHI(t-3),GHIs(t-3),Avg_TAR(t-3),Max_TAR(t-3),Min_TAR(t-3),TARGET(t-3),RH(t-2),Temp(t-2),Avg_temp(t-2),Max_temp(t-2),Min_temp(t-2),Avg_GHI(t-2),Max_GHI(t-2),Min_GHI(t-2),GHIs(t-2),Avg_TAR(t-2),Max_TAR(t-2),Min_TAR(t-2),TARGET(t-2),RH(t-1),Temp(t-1),Avg_temp(t-1),Max_temp(t-1),Min_temp(t-1),Avg_GHI(t-1),Max_GHI(t-1),Min_GHI(t-1),GHIs(t-1),Avg_TAR(t-1),Max_TAR(t-1),Min_TAR(t-1),TARGET(t-1)
144,0.679039,0.238889,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.0,0.244677,0.338365,0.064521,0.0,0.696678,0.233333,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.0,0.244677,0.338365,0.064521,0.0,0.672871,0.227778,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.0,0.244677,0.338365,0.064521,0.0,0.696461,...,0.0,0.480684,0.268519,0.199321,0.265217,0.167568,0.877605,0.940365,0.287738,0.0,0.434665,0.421102,0.150552,0.0,0.499188,0.270370,0.199321,0.265217,0.167568,0.877605,0.940365,0.287738,0.0,0.434665,0.421102,0.150552,0.0,0.494752,0.272222,0.199321,0.265217,0.167568,0.877605,0.940365,0.287738,0.0,0.434665,0.421102,0.150552,0.0
145,0.696678,0.233333,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.0,0.244677,0.338365,0.064521,0.0,0.672871,0.227778,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.0,0.244677,0.338365,0.064521,0.0,0.696461,0.220370,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.0,0.244677,0.338365,0.064521,0.0,0.669733,...,0.0,0.499188,0.270370,0.199321,0.265217,0.167568,0.877605,0.940365,0.287738,0.0,0.434665,0.421102,0.150552,0.0,0.494752,0.272222,0.199321,0.265217,0.167568,0.877605,0.940365,0.287738,0.0,0.434665,0.421102,0.150552,0.0,0.512282,0.274074,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.0,0.288665,0.315278,0.139784,0.0
146,0.672871,0.227778,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.0,0.244677,0.338365,0.064521,0.0,0.696461,0.220370,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.0,0.244677,0.338365,0.064521,0.0,0.669733,0.214815,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.0,0.244677,0.338365,0.064521,0.0,0.698409,...,0.0,0.494752,0.272222,0.199321,0.265217,0.167568,0.877605,0.940365,0.287738,0.0,0.434665,0.421102,0.150552,0.0,0.512282,0.274074,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.0,0.288665,0.315278,0.139784,0.0,0.507845,0.275926,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.0,0.288665,0.315278,0.139784,0.0
147,0.696461,0.220370,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.0,0.244677,0.338365,0.064521,0.0,0.669733,0.214815,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.0,0.244677,0.338365,0.064521,0.0,0.698409,0.205556,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.0,0.244677,0.338365,0.064521,0.0,0.672763,...,0.0,0.512282,0.274074,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.0,0.288665,0.315278,0.139784,0.0,0.507845,0.275926,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.0,0.288665,0.315278,0.139784,0.0,0.532626,0.277778,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.0,0.288665,0.315278,0.139784,0.0
148,0.669733,0.214815,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.0,0.244677,0.338365,0.064521,0.0,0.698409,0.205556,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.0,0.244677,0.338365,0.064521,0.0,0.672763,0.198148,0.168355,0.193478,0.224324,0.347313,0.698882,0.

In [78]:
testset9 = testset7.iloc[-48:,:] #season을 다시 붙인다면 testset7->8 근데 여기 이해가 안돼.. 48 뭐임? 하루치?
testset9

,RH(t-144),Temp(t-144),Avg_temp(t-144),Max_temp(t-144),Min_temp(t-144),Avg_GHI(t-144),Max_GHI(t-144),Min_GHI(t-144),GHIs(t-144),Avg_TAR(t-144),Max_TAR(t-144),Min_TAR(t-144),TARGET(t-144),RH(t-143),Temp(t-143),Avg_temp(t-143),Max_temp(t-143),Min_temp(t-143),Avg_GHI(t-143),Max_GHI(t-143),Min_GHI(t-143),GHIs(t-143),Avg_TAR(t-143),Max_TAR(t-143),Min_TAR(t-143),TARGET(t-143),RH(t-142),Temp(t-142),Avg_temp(t-142),Max_temp(t-142),Min_temp(t-142),Avg_GHI(t-142),Max_GHI(t-142),Min_GHI(t-142),GHIs(t-142),Avg_TAR(t-142),Max_TAR(t-142),Min_TAR(t-142),TARGET(t-142),RH(t-141),...,TARGET(t-4),RH(t-3),Temp(t-3),Avg_temp(t-3),Max_temp(t-3),Min_temp(t-3),Avg_GHI(t-3),Max_GHI(t-3),Min_GHI(t-3),GHIs(t-3),Avg_TAR(t-3),Max_TAR(t-3),Min_TAR(t-3),TARGET(t-3),RH(t-2),Temp(t-2),Avg_temp(t-2),Max_temp(t-2),Min_temp(t-2),Avg_GHI(t-2),Max_GHI(t-2),Min_GHI(t-2),GHIs(t-2),Avg_TAR(t-2),Max_TAR(t-2),Min_TAR(t-2),TARGET(t-2),RH(t-1),Temp(t-1),Avg_temp(t-1),Max_temp(t-1),Min_temp(t-1),Avg_GHI(t-1),Max_GHI(t-1),Min_GHI(t-1),GHIs(t-1),Avg_TAR(t-1),Max_TAR(t-1),Min_TAR(t-1),TARGET(t-1)
288,0.512282,0.274074,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.000000,0.288665,0.315278,0.139784,0.000000,0.507845,0.275926,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.000000,0.288665,0.315278,0.139784,0.000000,0.532626,0.277778,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.000000,0.288665,0.315278,0.139784,0.000000,0.522779,...,0.000000,0.536306,0.142593,0.093368,0.180435,0.091892,0.758337,0.928639,0.086222,0.000000,0.404037,0.425313,0.086034,0.000000,0.521805,0.144444,0.093368,0.180435,0.091892,0.758337,0.928639,0.086222,0.000000,0.404037,0.425313,0.086034,0.000000,0.512174,0.148148,0.093368,0.180435,0.091892,0.758337,0.928639,0.086222,0.000000,0.404037,0.425313,0.086034,0.000000
289,0.507845,0.275926,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.000000,0.288665,0.315278,0.139784,0.000000,0.532626,0.277778,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.000000,0.288665,0.315278,0.139784,0.000000,0.522779,0.281481,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.000000,0.288665,0.315278,0.139784,0.000000,0.550698,...,0.000000,0.521805,0.144444,0.093368,0.180435,0.091892,0.758337,0.928639,0.086222,0.000000,0.404037,0.425313,0.086034,0.000000,0.512174,0.148148,0.093368,0.180435,0.091892,0.758337,0.928639,0.086222,0.000000,0.404037,0.425313,0.086034,0.000000,0.484904,0.153704,0.260104,0.336957,0.224324,0.910587,0.951175,0.308772,0.000000,0.460685,0.437840,0.150545,0.000000
290,0.532626,0.277778,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.000000,0.288665,0.315278,0.139784,0.000000,0.522779,0.281481,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.000000,0.288665,0.315278,0.139784,0.000000,0.550698,0.283333,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.000000,0.288665,0.315278,0.139784,0.000000,0.546045,...,0.000000,0.512174,0.148148,0.093368,0.180435,0.091892,0.758337,0.928639,0.086222,0.000000,0.404037,0.425313,0.086034,0.000000,0.484904,0.153704,0.260104,0.336957,0.224324,0.910587,0.951175,0.308772,0.000000,0.460685,0.437840,0.150545,0.000000,0.476031,0.157407,0.260104,0.336957,0.224324,0.910587,0.951175,0.308772,0.000000,0.460685,0.437840,0.150545,0.000000
291,0.522779,0.281481,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.000000,0.288665,0.315278,0.139784,0.000000,0.550698,0.283333,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.000000,0.288665,0.315278,0.139784,0.000000,0.546045,0.285185,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.000000,0.288665,0.315278,0.139784,0.000000,0.574613,...,0.000000,0.484904,0.153704,0.260104,0.336957,0.224324,0.910587,0.951175,0.308772,0.000000,0.460685,0.437840,0.150545,0.000000,0.476031,0.157407,0.260104,0.336957,0.224324,0.910587,0.951175,0.308772,0.000000,0.460685,0.437840,0.150545,0.000000,0.452116,0.161111,0.260104,0.336957,0.224324,0.910587,0.951175,0.308772,0.000000,0.460685,0.437840,0.150545,0.000000
292,0.550698,0.283333,0

In [79]:
df_test = []

# 예시 코드
# 아직 유진이 어떻게 변수 추가할 지 몰라서 to_supervised는 쓰지 않았음.

for i in range(81):
    file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(i) + '.csv'
    new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(i) + '.csv'
    file_name = str(i) + '.csv'

    
    # adj_test에서 가져와야 하므로 new_path에서 불러온다
    temp = pd.read_csv(new_path)
    temp.drop('Unnamed: 0', axis = 1, inplace = True)
    testset = temp.copy()

    # 24로 저장한 24절기 season으로 이름 바꾸기
    testset.rename(columns = {'24' : 'season', 'T' : 'Temp'}, inplace = True)

    # 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
    testset2 = drop_clms(testset)
    testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
    testset3 = cos_time(testset2)
    testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'Temp', 'TARGET', 'sin_time', 'cos_time','season', 'Time']]

    testset5 = testset4.copy()
    testset5['GHIs'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)
    testset5 = add_variables(testset5)
    testset6 = testset5[['RH', 'Temp','Avg_temp', 'Max_temp', 'Min_temp', 'Avg_GHI', 'Max_GHI','Min_GHI', 'GHIs', 'Avg_TAR', 'Max_TAR', 'Min_TAR', 'TARGET']] #'DHI', 'DNI', 'season', 'SH', 'sin_time', 'cos_time',
    testset6 = pd.DataFrame(scaler.transform(testset6), columns=testset6.columns)
    testset7 = test_to_supervised(testset6, n_days)

#    testset8 = encoding(testset7)
#    testset8.drop('season', axis = 1, inplace = True)
    testset9 = testset7.iloc[-48:,:]

    df_test.append(testset9)

X_test = pd.concat(df_test)
# X_test = X_test.iloc[:, :n_obs]
X_test 

,RH(t-144),Temp(t-144),Avg_temp(t-144),Max_temp(t-144),Min_temp(t-144),Avg_GHI(t-144),Max_GHI(t-144),Min_GHI(t-144),GHIs(t-144),Avg_TAR(t-144),Max_TAR(t-144),Min_TAR(t-144),TARGET(t-144),RH(t-143),Temp(t-143),Avg_temp(t-143),Max_temp(t-143),Min_temp(t-143),Avg_GHI(t-143),Max_GHI(t-143),Min_GHI(t-143),GHIs(t-143),Avg_TAR(t-143),Max_TAR(t-143),Min_TAR(t-143),TARGET(t-143),RH(t-142),Temp(t-142),Avg_temp(t-142),Max_temp(t-142),Min_temp(t-142),Avg_GHI(t-142),Max_GHI(t-142),Min_GHI(t-142),GHIs(t-142),Avg_TAR(t-142),Max_TAR(t-142),Min_TAR(t-142),TARGET(t-142),RH(t-141),...,TARGET(t-4),RH(t-3),Temp(t-3),Avg_temp(t-3),Max_temp(t-3),Min_temp(t-3),Avg_GHI(t-3),Max_GHI(t-3),Min_GHI(t-3),GHIs(t-3),Avg_TAR(t-3),Max_TAR(t-3),Min_TAR(t-3),TARGET(t-3),RH(t-2),Temp(t-2),Avg_temp(t-2),Max_temp(t-2),Min_temp(t-2),Avg_GHI(t-2),Max_GHI(t-2),Min_GHI(t-2),GHIs(t-2),Avg_TAR(t-2),Max_TAR(t-2),Min_TAR(t-2),TARGET(t-2),RH(t-1),Temp(t-1),Avg_temp(t-1),Max_temp(t-1),Min_temp(t-1),Avg_GHI(t-1),Max_GHI(t-1),Min_GHI(t-1),GHIs(t-1),Avg_TAR(t-1),Max_TAR(t-1),Min_TAR(t-1),TARGET(t-1)
288,0.557299,0.353704,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.0,0.260408,0.389642,0.032259,0.0,0.580132,0.344444,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.0,0.260408,0.389642,0.032259,0.0,0.629477,0.337037,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.0,0.260408,0.389642,0.032259,0.0,0.634672,...,0.000000,0.805216,0.303704,0.308146,0.317391,0.362162,0.568941,0.833437,0.045658,0.0,0.314719,0.369770,6.451845e-02,0.0,0.798507,0.303704,0.308146,0.317391,0.362162,0.568941,0.833437,0.045658,0.0,0.314719,0.369770,6.451845e-02,0.0,0.805108,0.301852,0.308146,0.317391,0.362162,0.568941,0.833437,0.045658,0.0,0.314719,0.369770,6.451845e-02,0.0
289,0.580132,0.344444,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.0,0.260408,0.389642,0.032259,0.0,0.629477,0.337037,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.0,0.260408,0.389642,0.032259,0.0,0.634672,0.335185,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.0,0.260408,0.389642,0.032259,0.0,0.672113,...,0.000000,0.798507,0.303704,0.308146,0.317391,0.362162,0.568941,0.833437,0.045658,0.0,0.314719,0.369770,6.451845e-02,0.0,0.805108,0.301852,0.308146,0.317391,0.362162,0.568941,0.833437,0.045658,0.0,0.314719,0.369770,6.451845e-02,0.0,0.793529,0.300000,0.215979,0.178261,0.286486,0.065130,0.095574,0.010807,0.0,0.068938,0.062865,4.301463e-02,0.0
290,0.629477,0.337037,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.0,0.260408,0.389642,0.032259,0.0,0.634672,0.335185,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.0,0.260408,0.389642,0.032259,0.0,0.672113,0.333333,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.0,0.260408,0.389642,0.032259,0.0,0.672113,...,0.000000,0.805108,0.301852,0.308146,0.317391,0.362162,0.568941,0.833437,0.045658,0.0,0.314719,0.369770,6.451845e-02,0.0,0.793529,0.300000,0.215979,0.178261,0.286486,0.065130,0.095574,0.010807,0.0,0.068938,0.062865,4.301463e-02,0.0,0.800130,0.298148,0.215979,0.178261,0.286486,0.065130,0.095574,0.010807,0.0,0.068938,0.062865,4.301463e-02,0.0
291,0.634672,0.335185,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.0,0.260408,0.389642,0.032259,0.0,0.672113,0.333333,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.0,0.260408,0.389642,0.032259,0.0,0.672113,0.333333,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.0,0.260408,0.389642,0.032259,0.0,0.655557,...,0.000000,0.793529,0.300000,0.215979,0.178261,0.286486,0.065130,0.095574,0.010807,0.0,0.068938,0.062865,4.301463e-02,0.0,0.800130,0.298148,0.215979,0.178261,0.286486,0.065130,0.095574,0.010807,0.0,0.068938,0.062865,4.301463e-02,0.0,0.782599,0.296296,0.215979,0.178261,0.286486,0.065130,0.095574,0.010807,0.0,0.068938,0.062865,4.301463e-02,0.0
292,0.672113,0.333333,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.0,0.260408,0.389642,0.032259,0.0,0.672113,0.333333,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.0,0.260408,0.389642,0.032259,0

In [80]:
Y_train1 = pd.DataFrame(Y_train.iloc[:,0])
Y_train2 = pd.DataFrame(Y_train.iloc[:,1])

Y_valid1 = pd.DataFrame(Y_valid.iloc[:,0])
Y_valid2 = pd.DataFrame(Y_valid.iloc[:,1])

In [81]:
# [['RH', 'Temp','Avg_temp', 'Max_temp', 'Min_temp', 'Avg_GHI', 'Max_GHI','Min_GHI', 'SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']]

# RH_train = X_train.filter(regex='RH')
# T_train = X_train.filter(regex='Temp')
# # SH_train = X_train.filter(regex='SH')
# SH_train = X_train.filter(regex='SH').iloc[:,[0,48,96]]
# Avg_temp_train = X_train.filter(regex='Avg_temp').iloc[:,[0,48,96]]
# Max_temp_train = X_train.filter(regex='Max_temp').iloc[:,[0,48,96]]
# Min_temp_train = X_train.filter(regex='Min_temp').iloc[:,[0,48,96]]
# Avg_GHI_train = X_train.filter(regex='Avg_GHI').iloc[:,[0,48,96]]
# Max_GHI_train = X_train.filter(regex='Max_GHI').iloc[:,[0,48,96]]
# Min_GHI_train = X_train.filter(regex='Min_GHI').iloc[:,[0,48,96]]
# Avg_TAR_train = X_train.filter(regex='Avg_TAR').iloc[:,[0,48,96]]
# Max_TAR_train = X_train.filter(regex='Max_TAR').iloc[:,[0,48,96]]
# Min_TAR_train = X_train.filter(regex='Min_TAR').iloc[:,[0,48,96]]
# sin_train = X_train.filter(regex='sin_time').iloc[:,-6:]
# cos_train = X_train.filter(regex='cos_time').iloc[:,-6:]
# GHI_train = X_train.filter(regex='GHIs')
# TARGET_train = X_train.filter(regex='TARGET')



# RH_valid = X_valid.filter(regex='RH')
# T_valid = X_valid.filter(regex='Temp')
# # SH_valid = X_valid.filter(regex='SH')
# SH_valid = X_valid.filter(regex='SH').iloc[:,[0,48,96]]
# Avg_temp_valid = X_valid.filter(regex='Avg_temp').iloc[:,[0,48,96]]
# Max_temp_valid = X_valid.filter(regex='Max_temp').iloc[:,[0,48,96]]
# Min_temp_valid = X_valid.filter(regex='Min_temp').iloc[:,[0,48,96]]
# Avg_GHI_valid = X_valid.filter(regex='Avg_GHI').iloc[:,[0,48,96]]
# Max_GHI_valid = X_valid.filter(regex='Max_GHI').iloc[:,[0,48,96]]
# Min_GHI_valid = X_valid.filter(regex='Min_GHI').iloc[:,[0,48,96]]
# Avg_TAR_valid = X_valid.filter(regex='Avg_TAR').iloc[:,[0,48,96]]
# Max_TAR_valid = X_valid.filter(regex='Max_TAR').iloc[:,[0,48,96]]
# Min_TAR_valid = X_valid.filter(regex='Min_TAR').iloc[:,[0,48,96]]
# sin_valid = X_valid.filter(regex='sin_time').iloc[:,-6:]
# cos_valid = X_valid.filter(regex='cos_time').iloc[:,-6:]
# GHI_valid = X_valid.filter(regex='GHIs')
# TARGET_valid = X_valid.filter(regex='TARGET')

# RH_test = X_test.filter(regex='RH')
# T_test = X_test.filter(regex='Temp')
# # SH_test = X_test.filter(regex='SH')
# SH_test = X_test.filter(regex='SH').iloc[:,[0,48,96]]
# Avg_temp_test = X_test.filter(regex='Avg_temp').iloc[:,[0,48,96]]
# Max_temp_test = X_test.filter(regex='Max_temp').iloc[:,[0,48,96]]
# Min_temp_test = X_test.filter(regex='Min_temp').iloc[:,[0,48,96]]
# Avg_GHI_test = X_test.filter(regex='Avg_GHI').iloc[:,[0,48,96]]
# Max_GHI_test = X_test.filter(regex='Max_GHI').iloc[:,[0,48,96]]
# Min_GHI_test = X_test.filter(regex='Min_GHI').iloc[:,[0,48,96]]
# Avg_TAR_test = X_test.filter(regex='Avg_TAR').iloc[:,[0,48,96]]
# Max_TAR_test = X_test.filter(regex='Max_TAR').iloc[:,[0,48,96]]
# Min_TAR_test = X_test.filter(regex='Min_TAR').iloc[:,[0,48,96]]
# sin_test = X_test.filter(regex='sin_time').iloc[:,-6:]
# cos_test = X_test.filter(regex='cos_time').iloc[:,-6:]
# GHI_test = X_test.filter(regex='GHIs')
# TARGET_test = X_test.filter(regex='TARGET')

In [82]:
# GHI_train

In [106]:
# trainset = pd.concat([RH_train, T_train, sin_train, cos_train, GHI_train, TARGET_train, SH_train, Avg_GHI_train, Avg_temp_train, Max_GHI_train, Max_temp_train, Min_GHI_train, Min_temp_train, Avg_TAR_train, Max_TAR_train, Min_TAR_train], axis = 1)
# validset = pd.concat([RH_valid, T_valid, sin_valid, cos_valid, GHI_valid, TARGET_valid, SH_valid, Avg_GHI_valid, Avg_temp_valid, Max_GHI_valid, Max_temp_valid, Min_GHI_valid, Min_temp_valid, Avg_TAR_valid, Max_TAR_valid, Min_TAR_valid], axis = 1)
# testset = pd.concat([RH_test, T_test, sin_test, cos_test, GHI_test, TARGET_test, SH_test, Avg_GHI_test, Avg_temp_test, Max_GHI_test,Max_temp_test, Min_GHI_test, Min_temp_test, Avg_TAR_test, Max_TAR_test, Min_TAR_test], axis = 1)

In [107]:
# trainset

In [108]:
# validset

In [109]:
# testset

In [119]:
# LGBM 모델을 구축하고 예측까지 하는 함수를 만든다. 
def LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) 모델링
    model = LGBMRegressor(objective='quantile', alpha=q, max_depth = 25, num_leaves = 15, min_child_samples = 10,  min_child_weight = 150, min_split_gain = 0, n_estimators=20000, bagging_fraction=0.7, learning_rate=0.3, save_binary = True, subsample=0.9, is_unbalance = False, max_drop = 50, max_bin = 256, drop_rate  = 0.1, boosting_type = "gbdt", device = 'gpu')          
    
        # bagging_fraction : 배깅을 하기위해서 데이터를 랜덤 샘플링하여 학습에 사용한다. 비율은 0 < fraction <= 1 이며 0이 되지 않게 해야한다.
        # learning_rate : 일반적으로 0.01 ~ 0.1 정도로 맞추고 다른 파라미터를 튜닝한다. 나중에 성능을 더 높일 때 learning rate를 더 줄인다.
        # n_estimators : 기본값은 100, 1000 정도로 해주는 게 좋고 많을 수록 과적합이 발생한 가능성이 높음.
        # save_binary = True 넣어주면 더 빨라진다고 함.
        # https://greatjoy.tistory.com/72
        
    model.fit(X_train, Y_train, eval_metric = ['quantile'], eval_set=[(X_valid, Y_valid)], early_stopping_rounds=500, verbose=500)


        # verbose : eval metric이 이 숫자만큼의 round가 지난 다음 자동으로 출력된다.
        # early_stopping_rounds : 이 숫자가 가기 전까지 validation score가 증가하지 않으면 round를 멈춘다. 

    # (b) 예측
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

# LGBM 모델을 구축하고 예측까지 하는 함수를 만든다. 
def LGBM2(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) 모델링
    model = LGBMRegressor(objective='quantile', alpha=q, max_depth = 25, num_leaves = 15, min_child_samples = 10,  min_child_weight = 150, min_split_gain = 0, n_estimators=15000, bagging_fraction=0.7, learning_rate=0.005, save_binary = True, subsample=0.9, is_unbalance = False, max_drop = 50, max_bin = 256, drop_rate  = 0.1, boosting_type = "gbdt", device = 'gpu')          
    
        # bagging_fraction : 배깅을 하기위해서 데이터를 랜덤 샘플링하여 학습에 사용한다. 비율은 0 < fraction <= 1 이며 0이 되지 않게 해야한다.
        # learning_rate : 일반적으로 0.01 ~ 0.1 정도로 맞추고 다른 파라미터를 튜닝한다. 나중에 성능을 더 높일 때 learning rate를 더 줄인다.
        # n_estimators : 기본값은 100, 1000 정도로 해주는 게 좋고 많을 수록 과적합이 발생한 가능성이 높음.
        # save_binary = True 넣어주면 더 빨라진다고 함.
        # https://greatjoy.tistory.com/72
        
    model.fit(X_train, Y_train, eval_metric = ['quantile'], eval_set=[(X_valid, Y_valid)], early_stopping_rounds=500, verbose=500)


        # verbose : eval metric이 이 숫자만큼의 round가 지난 다음 자동으로 출력된다.
        # early_stopping_rounds : 이 숫자가 가기 전까지 validation score가 증가하지 않으면 round를 멈춘다. 

    # (b) 예측
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [120]:
# Target 예측

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    for q in quantiles:
        print(q)
        pred, model = LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)

    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred

# Target 예측

def train_data2(X_train, Y_train, X_valid, Y_valid, X_test):

    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    for q in quantiles:
        print(q)
        pred, model = LGBM2(q, X_train, Y_train, X_valid, Y_valid, X_test)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)

    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred

In [117]:
qq = 0
quantiles = [0.1-qq, 0.2-qq, 0.3-qq, 0.4-qq, 0.5, 0.6-qq, 0.7-qq, 0.8-qq, 0.9-qq]

In [121]:
tick = time.time()

# Target1

models_1, results_1 = train_data(X_train, Y_train1, X_valid, Y_valid1, X_test)
results_1.sort_index()[:48]

0.1
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.16887
[1000]	valid_0's quantile: 1.156
[1500]	valid_0's quantile: 1.15261
[2000]	valid_0's quantile: 1.15037
[2500]	valid_0's quantile: 1.14905
[3000]	valid_0's quantile: 1.148
Early stopping, best iteration is:
[2948]	valid_0's quantile: 1.14778
0.2
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.9 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.72353
[1000]	valid_0's quantile: 1.68609
[1500]	valid_0's quantile: 1.67884
[2000]	valid_0's quantile: 1.67164
[2500]	valid_0's quantile: 1.66708
[3000]	valid_0's quantile: 1.66388
[3500]	valid_0's quantile: 1.66099
[4000]	valid_0's quantile: 1.65882
[4500]	valid_0's quantile: 1.6575
[5000]	valid_0's quantile: 1.65598
[5500]	valid_0's quantile: 1.65485
[6000]	valid_0's quantile: 1.65436
[6500]	valid_0's quantile: 1.65335
[7000]	valid_0's quanti

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [122]:
# Target2

models_2, results_2 = train_data2(X_train, Y_train2, X_valid, Y_valid1, X_test)
tock = time.time()
results_2.sort_index()

0.1
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.9 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.43877
[1000]	valid_0's quantile: 1.41344
[1500]	valid_0's quantile: 1.40806
[2000]	valid_0's quantile: 1.40728
[2500]	valid_0's quantile: 1.40746
Early stopping, best iteration is:
[2133]	valid_0's quantile: 1.40694
0.2
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.9 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 2.37137
[1000]	valid_0's quantile: 2.29017
[1500]	valid_0's quantile: 2.26555
[2000]	valid_0's quantile: 2.2549
[2500]	valid_0's quantile: 2.25174
[3000]	valid_0's quantile: 2.25133
[3500]	valid_0's quantile: 2.24814
Early stopping, best iteration is:
[3465]	valid_0's quantile: 2.24801
0.3
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.9 will be i

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3885,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
print('소요시간 :', tock-tick)

print(results_1.shape, results_2.shape)

소요시간 : 6646.680105686188
(3888, 9) (3888, 9)


In [124]:
# 출력

submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission2 = submission.set_index('id')
submission3 = not_minus(submission2)

In [148]:
def columns_IDA(dataset):
  dataset2 = pd.DataFrame()
  for j in range(0,9):
    new_val = []
    for m in range(1, int(dataset.shape[0]/48)+1):
      temp = dataset.iloc[48*(m-1):48*m, j].values

      strt = 0
      endd = 0
      k = 0

      for i in range(len(temp)):
        if temp[i] > k and k ==0 and i < 24:
          strt = i
        elif temp[i] < k and temp[i] == 0 and i > 24:
          endd = i
        k = temp[i]

      bf = temp[:strt+1]
      y = temp[strt+1:endd-1]
      af = temp[endd-1:]
      # bf = temp[:strt]
      # y = temp[strt:endd]
      # af = temp[endd:]

      x = np.array([i for i in range(len(y))])
      fit_t = np.polyfit(x,y,5)

      for i in x:
        fit1 = fit_t[0]*x**5 + fit_t[1]*x**4 + fit_t[2]*x**3 + fit_t[3]*x**2 + fit_t[4]*x + fit_t[5]
    # fit1 = fit_t[0]*x**4 + fit_t[1]*x**3 + fit_t[2]*x**2 + fit_t[3]*x + fit_t[4]
    # fit1 = fit_t[0]*x**2 + fit_t[1]*x + fit_t[2]

      revised = np.concatenate((bf, fit1, af), axis=0)
      new_val.extend(list(revised))
    k = pd.DataFrame(new_val)
    dataset2 = pd.concat([dataset2, k], axis = 1)
    print(j+1,'/9')
  return dataset2

def f(x, a,b,c, d):
  y = 1/3 * b * x**3 - a*b *x**2 + (a**2*b + c**2)*x + d
  # y = a*(x-12*a) * x **2 + c
  # y = a*x**2 + b
  return y

def rows_IDA(dataset):
  revised = pd.DataFrame()
  for i in range(dataset.shape[0]):
    y = dataset.iloc[i,:].values
    y = np.array(list(y))
    x = np.array([i for i in range(len(y))])

    if list(y).count(0) <= 5 :
      popt, pcov = curve_fit(f, x, y, maxfev = 999990000)
      yfit = f(x, *popt)
      revised = pd.concat([revised, pd.DataFrame(yfit).transpose()], axis = 0)
    else:
      revised = pd.concat([revised, pd.DataFrame(y).transpose()], axis = 0)
    if i % 400 == 0:
      print(i, '/7776')
  return revised

In [151]:
submission4 = columns_IDA(submission3)
submission4 = not_minus(submission4)
submission5 = pd.DataFrame(submission4.values, columns = submission3.columns, index = submission3.index)

In [152]:
submission5

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
id,,,,,,,,,
0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [155]:
test = rows_IDA(submission5)
submission6 = not_minus(test)

0 /7776


/usr/local/lib/python3.6/dist-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


100 /7776
200 /7776
300 /7776
400 /7776
500 /7776
600 /7776
700 /7776
800 /7776
900 /7776
1000 /7776
1100 /7776
1200 /7776
1300 /7776
1400 /7776
1500 /7776
1600 /7776
1700 /7776
1800 /7776
1900 /7776
2000 /7776
2100 /7776
2200 /7776
2300 /7776
2400 /7776
2500 /7776
2600 /7776
2700 /7776
2800 /7776
2900 /7776
3000 /7776
3100 /7776
3200 /7776
3300 /7776
3400 /7776
3500 /7776
3600 /7776
3700 /7776
3800 /7776
3900 /7776
4000 /7776
4100 /7776
4200 /7776
4300 /7776
4400 /7776
4500 /7776
4600 /7776
4700 /7776
4800 /7776
4900 /7776
5000 /7776
5100 /7776
5200 /7776
5300 /7776
5400 /7776
5500 /7776
5600 /7776
5700 /7776
5800 /7776
5900 /7776
6000 /7776
6100 /7776
6200 /7776
6300 /7776
6400 /7776
6500 /7776
6600 /7776
6700 /7776
6800 /7776
6900 /7776
7000 /7776
7100 /7776
7200 /7776
7300 /7776
7400 /7776
7500 /7776
7600 /7776
7700 /7776


In [158]:
submission7 = pd.DataFrame(submission6.values, columns = submission3.columns, index = submission3.index)
submission7

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
id,,,,,,,,,
0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [159]:
submission7.to_csv('multivar.csv')
!cp multivar.csv "drive/My Drive/"